In [1]:
import pandas as pd
import numpy as np
import ruptures as rpt
from sklearn.linear_model import LinearRegression


true_CP = [51,101,151]
T = 200
def evaluate_change_points(est_CP, true_CP, T):
    tau = T - 1
    num_CP = len(est_CP)

    if num_CP == 0:
        dist_est_gt = np.inf
        dist_gt_est = -np.inf
        covering_metric = 0
    else:
        # max over true CPs of distance to nearest est CP
        holder_est_gt = [min(abs(np.array(est_CP) - i)) for i in true_CP]
        dist_est_gt = max(holder_est_gt) if holder_est_gt else np.inf

        # max over est CPs of distance to nearest true CP
        holder_gt_est = [min(abs(np.array(true_CP) - i)) for i in est_CP]
        dist_gt_est = max(holder_gt_est) if holder_gt_est else np.inf

        # add boundaries
        gt_CP_all = [1] + list(true_CP) + [T + 1]
        est_CP_all = [1] + list(est_CP) + [T + 1]

        # construct segment index sets
        gt_list = [set(range(gt_CP_all[i-1], gt_CP_all[i])) for i in range(1, len(gt_CP_all))]
        est_list = [set(range(est_CP_all[i-1], est_CP_all[i])) for i in range(1, len(est_CP_all))]

        covering_metric = 0
        for A in gt_list:
            jaccard_vals = []
            for Ap in est_list:
                inter = len(A & Ap)
                union = len(A | Ap)
                jaccard_vals.append(inter / union if union > 0 else 0)
            covering_metric += len(A) * max(jaccard_vals)
        covering_metric /= (tau + 1)

    abs_error = abs(num_CP - len(true_CP))

    return pd.DataFrame([{
        "abs_error": abs_error,
        "dist_est_gt": dist_est_gt,
        "dist_gt_est": dist_gt_est,
        "covering_metric": covering_metric,
        "est_CP": est_CP
    }])

def auto_select_cps(resid_norm, Kmax=10, model="l2"):
    """
    Automatic CP selection via BIC
    """
    T = len(resid_norm)
    signal = resid_norm.reshape(-1, 1)

    best_k, best_bic, best_bkps = 0, np.inf, [T]
    for k in range(Kmax+1):
        if k == 0:
            mean_val = np.mean(signal)
            ssr = np.sum((signal - mean_val) ** 2)
            bic = T * np.log(ssr / T) if ssr > 0 else -np.inf
            bkps = [T]
        else:
            algo = rpt.Binseg(model=model).fit(signal)
            bkps = algo.predict(n_bkps=k)
            ssr = 0
            start = 0
            for b in bkps:
                seg = signal[start:b]
                mean_val = np.mean(seg)
                ssr += np.sum((seg - mean_val) ** 2)
                start = b
            bic = T * np.log(ssr / T) + (k+1) * np.log(T)

        if bic < best_bic:
            best_bic, best_k, best_bkps = bic, k, bkps

    est_CP = [b + 1 for b in best_bkps[:-1]]
    return est_CP, best_k, best_bic


reps = 20
results_list = []
for rep in range(reps):
    dat_y = pd.read_csv(f"./reps_sim_dat/y_rep{rep}.csv").values
    dat_x = pd.read_csv(f"./reps_sim_dat/x_rep{rep}.csv").values
    
    T, N, dy = 200, 100, 3
    dx = 3

    dat_y_T = dat_y.T 
    Y_array = dat_y_T.reshape(dy, N, T, order="F").transpose(2,1,0)  # (T, N, dy)
    dat_x_T = dat_x.T
    X_array = dat_x_T.reshape(dx, N, T, order="F").transpose(2,1,0)  # (T, N, dx)
    Y_stack = Y_array.reshape(T * N, dy)
    X_stack = X_array.reshape(T * N, dx)

    residuals_mat = np.zeros((T * N, dy))
    for j in range(dy):
        model = LinearRegression().fit(X_stack, Y_stack[:, j])
        y_pred = model.predict(X_stack)
        residuals_mat[:, j] = Y_stack[:, j] - y_pred

    time_index = np.repeat(np.arange(T), N)
    resid_norm = np.array([
        np.sum(residuals_mat[time_index == t] ** 2)
        for t in range(T)
    ])
    # print(resid_norm)
    # Method1: Ruptures
    # Truong, C., Oudre, L., & Vayatis, N. (2018). ruptures: change point detection in Python. arXiv preprint arXiv:1801.00826.
    rup_est_CP, k_hat, bic_val = auto_select_cps(resid_norm, Kmax=8)
    res_auto = evaluate_change_points(rup_est_CP, true_CP, T)
    res_auto["rep"] = rep
    res_auto["method"] = "ruptures"
    results_list.append(res_auto)

ValueError: cannot reshape array of size 59997 into shape (3,100,200)

In [10]:
results_list

[   abs_error  dist_est_gt  dist_gt_est  covering_metric               est_CP  \
 0          1            0           15            0.925  [51, 101, 151, 166]   
 
    rep    method  
 0    0  ruptures  ,
    abs_error  dist_est_gt  dist_gt_est  covering_metric  \
 0          4            0           40            0.875   
 
                               est_CP  rep    method  
 0  [11, 51, 101, 151, 156, 161, 166]    1  ruptures  ,
    abs_error  dist_est_gt  dist_gt_est  covering_metric  \
 0          3            0           35            0.825   
 
                           est_CP  rep    method  
 0  [51, 101, 151, 161, 171, 186]    2  ruptures  ,
    abs_error  dist_est_gt  dist_gt_est  covering_metric  \
 0          3            5           40              0.8   
 
                         est_CP  rep    method  
 0  [11, 31, 56, 101, 106, 151]    3  ruptures  ,
    abs_error  dist_est_gt  dist_gt_est  covering_metric  \
 0          2            0           25            0.875

In [ ]:

df = pd.read_pickle("exp_results.pkl")

df_new = df.copy()
df_new["res_full_clean"] = df_new["res_full"].apply(lambda x: tuple(x[:-1])) 


res_expanded = pd.DataFrame(df_new["res_full_clean"].tolist(),
                            columns=["abs_error", "dist_est_gt", "dist_gt_est", "covering_metric", "score2", "est_cp"])
res_expanded["rep"] = res_expanded["rep"] = range(len(res_expanded))
res_expanded["method"] = "Allen"
res_expanded = res_expanded.drop(columns=["score2"])
res_expanded = res_expanded.rename(columns={"est_cp": "est_CP"})
print(res_expanded)


In [36]:
res_ruptures = pd.concat(results_list, ignore_index=True)
final_df = pd.concat([res_expanded, res_ruptures], ignore_index=True)
print(final_df)
final_df.to_csv("exp_results_combined.csv", index=False)

    abs_error  dist_est_gt  dist_gt_est  covering_metric  \
0           0            0            0         1.000000   
1           1           50            0         0.750000   
2           2          100            0         0.500000   
3           1           50           20         0.578571   
4           0           50           40         0.700000   
5           0            0            0         1.000000   
6           4            8           35         0.594815   
7           1           50            0         0.750000   
8           0            0            0         1.000000   
9           1           50            0         0.750000   
10          0            0            0         1.000000   
11          0            0            0         1.000000   
12          0            0            0         1.000000   
13          1           50            0         0.750000   
14          0            0            0         1.000000   
15          0           50           43 